# Identifying Potential Protein-Coding Genes in the SARS-CoV-2 Virus

Author: **Daniel Park**

The SARS-CoV-2 Virus, or more commonly known as COVID-19 or the coronavirus, an infectious disease caused by a newly discovered coronavirus. Most people infected with the COVID-19 Virus will experience mild to moderate respiratory illness and recover without requiring special treatment. Older people and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and cancer are more likely to develop severe illness.

Our bioinformatics analysis will identify potential protein-coding genes in the SARS-CoV-2 Virus and match these genes to known viral proteins. These proteins are all significant because we can use them to get a better understanding of these proteins can give us insights into how the Virus works and possible routes to treatment or a vaccine.

## Setting up the Analysis

To ensure that the plots display correctly in this document, run the code cell below to reload modules before executing user code to make workflow possible.

In [1]:
%load_ext autoreload
%autoreload 2

After reloading the document, we can import the implementation used to analyze a FASTA file under the name "NC_045512.2.fa" that contains the nucleotide sequence of the SARS-CoV-2 virus (specifically, the isolate Wuhan-Hu-1 strain of the virus, which was the first to be sequenced). 

The cell below will import all the implementation functions under the file "gene_finder.py" and "helpers.py."

In [13]:
from gene_finder import *
from helpers import *

## Gene Finding Analysis Implementation

In order to perform an analysis of the SARS-CoV-2 genome, the `main` function will call `find_genes` that will find the potential protein-coding genes and twelve `helper` functions (listed below) that I will either use to help other helper functions or call them directly in the primary function 'find_genes' for analysis.

The overall process to get our protein-coding genes is to find ORFs, or open reading frames, in a DNA strand that can be translated into proteins. We will find all the possible ORFs within a strand of DNA by randomly shuffling our strand and searching for ORFs, including in-frame ORFs and ORFs found one or two nucleotides from the start of the DNA strand. The ORFs found in the strand will be translated into amino acids that show us all the potential protein-coding genes.

Once the candidate genes are identified, they can be looked up in [protein-BLAST](http://www.ncbi.nlm.nih.gov/blast/Blast.cgi?PROGRAM=blastp&PAGE_TYPE=BlastSearch&LINK_LOC=blasthome) to find information about five significant proteins in the DNA sequence.

### Helper Functions

#### get_complement

The function `get_complement` takes a string nucleotide consisting of a single character of `A, T, C, or G`, each representing the DNA nucleotides adenine, thymine, cytosine, and guanine. The function will return a string (also consisting of a single character) representing the complementary nucleotide. In DNA, `A and T` complement each other, as are `C and G.`

In [7]:
def get_complement(nucleotide):
    complement = ""
    if nucleotide in 'A':
        complement = 'T'
    elif nucleotide in 'T':
        complement = 'A'
    elif nucleotide in 'G':
        complement = 'C'
    else:
        complement = 'G'
    return complement

#### get_reverse_complement

The function `get_reverse_complement` takes a string strand representing a single strand of DNA (consisting entirely of the characters `A, T, C, and G`)and returns a string representing a complementary strand of DNA. The `get_complement` function mentioned before will get a nucleotide from the strand and compute the strand's reversal inside the function.

In [8]:
def get_reverse_complement(strand):
    reverse_sequence = ''
    for base in reversed(strand):
        reverse_sequence += get_complement(base)
    return reverse_sequence

#### rest_of_orf

An open reading frame (ORF) is a sequence of nucleotides in a DNA strand that can be translated into a protein, which will be a crucial component of identifying genes of interest in DNA. 

The function 'rest_of_orf' takes a string strand representing a strand of DNA that begins with a start codon and returns the sequence of nucleotides representing the rest of the ORF, up to but not including the next stop codon, which includes `'TAA, 'TAG', 'TGA.'`

In [9]:
def rest_of_orf(strand):
    strand_list = [strand[nucleotide:nucleotide+3] for nucleotide in range(0, len(strand), 3)]

    for codon_index, codon in enumerate(strand_list):
        if amino_acid(codon) == "*":
            strand_list = strand_list[0:codon_index]
            break

    return ''.join(strand_list)

#### find_all_orfs_one_frame

The function `find_all_orfs_one_frame` will take a string `strand` representing a strand of DNA and return a list of strings representing all the ORFS in one frame, which means that each ORF should be a multiple of three nucleotides from the start of the strand. The`rest_of_orf` function is called within this function to find all the ORFs to help me get all the possible ORFs within the string `strand.`

In [10]:
def find_all_orfs_one_frame(strand):
    frame = ""
    in_frame_orfs = []
    
    while len(strand) >= 3:
        current_codon = strand[:3]
        if current_codon != "ATG":
            strand = strand[3:]
            continue
        frame = rest_of_orf(strand)
        strand = strand[len(frame):]
        in_frame_orfs.append(frame)

    return in_frame_orfs

#### find_all_orfs

The function `find_all_orfs` will have a similar process to `find_all_orfs_one_frame`, but instead of only searching for ORFs within the same frame, the function will find all the ORFs within different frames as well, meaning that `strand` taken in by the function will be shifted by one and two nucleotides as well.

Below is an example of how frameshifting will occur:

```
ATG AAA ATG GCA TGA  <- In-frame
A TGA AAA TGG CAT GA  <- Frame-shifted by one nucleotide
AT GAA AAT GGC ATG A  <- Frame-shifted by two nucleotides
```

I used the previous function `find_all_orfs_one_frame` to help me find all the ORFs when the strand is frame-shifted 

In [11]:
def find_all_orfs(strand):
    all_orfs = []

    for start in range(3):
        all_orfs += find_all_orfs_one_frame(strand[start:])

    return all_orfs

#### find_all_orfs_both_strands

The function `find_all_orfs_both_strands` will complete the same process as `find_all_orfs` by searching for all ORFs within different frames with both the original `strand` and its reverse complement. 

In [12]:
def find_all_orfs_both_strands(strand):
    all_frame_orfs_both_strands = []

    for _ in range(2):
        all_frame_orfs_both_strands.append(find_all_orfs(strand))
        strand = get_reverse_complement(strand)

    all_frame_orfs_both_strands = [
        orf for orf_list in all_frame_orfs_both_strands for orf in orf_list]

    return all_frame_orfs_both_strands

#### find_longest_orf

This function will find the longest ORF in that list with the list of ORFs found in `find_all_orfs_both_strands`.

However, instead of having the list as our input, the string `strand` is the input. Calling the `find_all_orfs_both_strands` function in `find_longest_orf` can run this function on its own without having to call `find_all_orfs_both_strands` before this function to have an output.

In [14]:
def find_longest_orf(strand):
    list_of_orfs = find_all_orfs_both_strands(strand)
    longest_orf_length = 0
    longest_orf = ""

    for orf in list_of_orfs:
        if len(orf) > longest_orf_length:
            longest_orf_length = len(orf)
            longest_orf = orf
    return longest_orf

#### shuffle

The function `shuffle` will shuffle nucleotides' order in a strand of DNA and return the shuffled DNA strand.

In [17]:
def shuffle(strand):
    return "".join(random.sample(strand, len(strand)))

#### noncoding_orf_threshold

The function `noncoding_orf_threshold` filters out ORFs that are too short of producing any valuable proteins and filters them out.

The function takes a string `strand` representing a strand of DNA and a positive integer `num_trials` representing a specific amount of trials to run, and randomly shuffles the nucleotides in the strand for each trial and finds the longest ORF in either the shuffled strand or its reverse complement. The function keeps track of the minimum length of this value over all trials and returns an integer representing this minimum length.

In [16]:
def noncoding_orf_threshold(strand, num_trials):
    orfs_lengths = []

    for _ in tqdm(range(num_trials)):
        # shuffles the strand
        shuffled_strand = shuffle(strand)

        # finds the longest ORF in either the shuffled strand or its reverse complement.
        longest_orf = find_longest_orf(shuffled_strand)

        # keeps track of the minimum length of this value over all of the trials
        length_of_longest_orf = len(longest_orf)

        # returns an integer representing this minimum length
        orfs_lengths.append(length_of_longest_orf)

    # returns an integer representing this minimum length
    return min(orfs_lengths)

#### amino_acids

The function `amino_acids` returns the amino acid symbol corresponding to the DNA codon.

In [18]:
def amino_acid(codon): 
    # Some amino acids can be determined solely by the first two nucleotides.
    if codon[:2] in PARTIAL_CODON_TABLE:
        return PARTIAL_CODON_TABLE[codon[:2]]

    # Many other amino acids can be determined by the first two nucleotides,
    # plus whether the last nucleotide is a purine (A/G) or pyramidine (T/C).
    if codon[:2] in BRANCHED_CODON_TABLE:
        branches = BRANCHED_CODON_TABLE[codon[:2]]
        if codon[-1] in PURINES:
            return branches[0]
        return branches[1]

    # The few amino acids left can be handled on a case-by-case basis.
    if codon == "ATG":
        return "M"  # Methionine/START
    if codon[:2] == "AT":
        return "I"  # Isoleucine
    # At this point we know the first two characters of the codon are "TG".
    if codon[-1] not in PURINES:
        return "C"  # Cysteine
    if codon[-1] == "A":
        return "*"  # STOP
    return "W"  # Tryptophan

#### encode_amino_acids

Using the function `amino_acids`, I can change a DNA strand into its amino acid form. 

The function `encode_amino_acids` takes a string `orf` representing a strand of DNA that is an ORF and returns a string representing the sequence of amino acids, with each amino acid written as its one-letter symbol.

In [19]:
def encode_amino_acids(orf):
    amino_acids = ""

    for triple in range(0, len(orf), 3):
        codon = orf[triple:triple + 3]
        amino_acids += amino_acid(codon)

    return amino_acids

#### load_fasta_file

The function `load_fasta_file` will read and return a sequence of DNA nucleotides from a FASTA file.

In bioinformatics and biochemistry, a FASTA file format contains nucleotide or amino acid sequences where nucleotides or amino acids are represented using single-letter codes. 

In [20]:
def load_fasta_file(path):
    """
    Read and return a sequence of DNA nucleotides from a FASTA file.

    Args:
        path: A string representing the path to the FASTA file.

    Returns:
        A string representing the sequence of DNA nucleotides (i.e., the
        characters A, T, C, or G) in the FASTA file.
    """
    sequence = ""
    read_header = False
    with open(path, "r") as fasta_file:
        for line in fasta_file:
            if not read_header:
                # Skip the header line.
                read_header = True
                continue
            sequence += line.strip()
    return sequence

### Main Function

With all the helper functions implemented, I put everything together to find potential protein-coding genes.

The function `find_genes` will take in a string `path` that represents the FASTA file's location with the SARCS-CoV-2 sequence and load it into a string of nucleotides. Next, I will find the threshold length to use as a cutoff for coding ORFs, using 1,500 trials of shuffling the nucleotide sequence. Then, I will find all the ORFs in both strands of the sequence. In all those ORFs, I will translate the ORF to a sequence of amino acids for any ORFs longer than the threshold length.

In [21]:
def find_genes(path):
    # to be returned
    amino_acid_sequences = []

    # Loading the string of nucleotides from the file.
    genes = load_fasta_file(path)

    # Determining the threshold length to use as a cutoff
    # for coding ORFs, using 1,500 trials of shuffling
    # the nucleotide sequence.
    threshold_length = noncoding_orf_threshold(genes, 1500)

    # Find all ORFs in both strands of the sequence.
    all_orfs_list = find_all_orfs_both_strands(genes)

    # For any ORF longer than the cutoff length,
    # translate the ORF to a sequence of amino acids.
    for orf in tqdm(all_orfs_list):
        if len(orf) > threshold_length:
            protein_orf = encode_amino_acids(orf)
            amino_acid_sequences.append(protein_orf)

    return amino_acid_sequences

## Identifying Potential Protein-Coding Genes

After the implementation, I ran the `find_genes` function to identify the potential protein-coding genes in the SARS-CoV-2 Virus.

Run the cell below to get a list of candidate genes that could be potential proteins that can give us insights into how the Virus works and possible routes to treatment or a vaccine. Fair warning, the code below may take a while, and there should be a progress bar that shows how much longer the function must run in order to be complete.

In [22]:
find_genes("data/NC_045512.2.fa")

<ipython-input-16-4255bd2c1694>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm(range(num_trials)):


<ipython-input-21-7db4be4b6d2c>:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for orf in tqdm(all_orfs_list):


['MVPHISRQRLTKYTMADLVYALRHFDEGNCDTLKEILVTYNCCDDDYFNKKDWYDFVENPDILRVYANLGERVRQALLKTVQFCDAMRNAGIVGVLTLDNQDLNGNWYDFGDFIQTTPGSGVPVVDSYYSLLMPILTLTRALTAESHVDTDLTKPYIKWDLLKYDFTEERLKLFDRYFKYWDQTYHPNCVNCLDDRCILHCANFNVLFSTVFPPTSFGPLVRKIFVDGVPFVVSTGYHFRELGVVHNQDVNLHSSRLSFKELLVYAADPAMHAASGNLLLDKRTTCFSVAALTNNVAFQTVKPGNFNKDFYDFAVSKGFFKEGSSVELKHFFFAQDGNAAISDYDYYRYNLPTMCDIRQLLFVVEVVDKYFDCYDGGCINANQVIVNNLDKSAGFPFNKWGKARLYYDSMSYEDQDALFAYTKRNVIPTITQMNLKYAISAKNRARTVAGVSICSTMTNRQFHQKLLKSIAATRGATVVIGTSKFYGGWHNMLKTVYSDVENPHLMGWDYPKCDRAMPNMLRIMASLVLARKHTTCCSLSHRFYRLANECAQVLSEMVMCGGSLYVKPGGTSSGDATTAYANSVFNICQAVTANVNALLSTDGNKIADKYVRNLQHRLYECLYRNRDVDTDFVNEFYAYLRKHFSMMILSDDAVVCFNSTYASQGLVASIKNFKSVLYYQNNVFMSEAKCWTETDLTKGPHEFCSQHTMLVKQGDDYVYLPYPDPSRILGAGCFVDDIVKTDGTLMIERFVSLAIDAYPLTKHPNQEYADVFHLYLQYIRKLHDELTGHMLDMYSVMLTNDNTSRYWEPEFYEAMYTPHTVLQAVGACVLCNSQTSLRCGACIRRPFLCCKCCYDHVISTSHKLVLSVNPYVCNAPGCDVTDVTQLYLGGMSYYCKSHKPPISFPLCANGQVFGLYKNTCVGSDNVTDFNAIATCDWTNAGDYILANTCTERLKLFAAETLKATEETFKLSYGIATVREVLSDRELHLSWEVGKPRPP

The output above may seem daunting at first, but to break it down into simple terms, the output shows us a list of eighteen possible proteins we can decode. Running each protein from the list through the [protein-BLAST](http://www.ncbi.nlm.nih.gov/blast/Blast.cgi?PROGRAM=blastp&PAGE_TYPE=BlastSearch&LINK_LOC=blasthome) tool, we can narrow the eighteen possible proteins into five major proteins:

1. **Polyprotein ORF1a:** a large "precursor protein" from which many of the virus's proteins are formed. This is done by cutting or cleaving the polyprotein at certain locations.
    1. Amino Acid Sequence: ![Poly Protein ORF1a](images/polyprotein_orf1a.png)
    
    2. Length of the Protein: 2595 amino acids
    
    3. Accession Number: QJW69675.1
    
2. **Nucleocapsid Protein**: carries the virus's genetic material.
    1. Amino Acid Sequence: ![Nucleocapsid Protein](images/nucleocapsid_protein.png)
    2. Length of the Protein: 419 amino acids
    3. Accession Number: QRV71356.1
    
3. **Envelope Protein**: forms part of the outer layer of the virus, and protects it from the host's immune system as the virus travels between host cells.
    1. Amino Acid Sequence: MYSFVSEETGTLIVNSVLLFLAFVVFLLVTLAILTALRLCAYCCNIVNVSLVKPSFYVYSRVKNLNSSRVPDLLV
    2. Length of the Protein: 75 amino acids
    3. Accession Number: QPK75943.1
    
4. **Membrane Protein**: part of the outer layer of the virus that fuses with the host cell's membrane (the cell's outer layer) when the virus enters the host cell.
    1. Amino Acid Sequence: ![Membrane Protein](images/membrane_protein.png)
    2. Length of the Protein: 222 amino acids
    3. Accession Number: YP_009724393.1
    
5. **Spike Protein:** binds with specific receptors on the host cell, starting the infection of the host cell. The spike protein ensures that the virus only infects the type(s) of host cells that it is suited for.
    1. Amino Acid Sequence: ![Spike Protein](images/spike_protein.png)
    2. Length of the Protein: 1282 amino acids
    3. Accession Number: BCN86353.1

This information is beneficial since we know the specific amino acid sequence, length, and accession number. We can create a treatment or vaccination that could potentially target these proteins. For example, treatment would block a spike protein's ability to bind with and infect the proper host cell, reducing the virus's chances of infecting a person.